# Passive Solar Analysis

The goal of this notebook is to explore and better understand general principles associated with passive solar energy implementations.

![Overview Image](https://github.com/d33bs/passive_solar_analysis/blob/main/overview.svg?raw=true)

## Resources
- Wikipedia page covering Passive Solar Building Design https://en.wikipedia.org/wiki/Passive_solar_building_design
- New Mexico Solar Energy Association (NMSEA) Overview https://www.nmsolar.org/passive-solar/
- NMSEA Energy 101 https://www.nmsolar.org/energy-101/
- NMSEA Passive Solar Design https://www.nmsolar.org/wp-content/uploads/2017/04/Passive-Solar-Design.pdf